In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

# Imports
import configparser
import pynattas as pnas
from datetime import datetime
import time

config = configparser.ConfigParser()
config.read('config.ini')


nas_check = config.getboolean(section='Mode', option='network_architecture_search')
ht_check = config.getboolean(section='Mode', option='hyperparameter_tuning')


task = config['Mode']['task']
if task not in ['C','D']:
    print(f"Error: Selected Task {task} is not available. Check config_optimizer.ini for verification.")
    exit()
else:
    print(f"Task: {task}")

Task: C


In [3]:
# Define NAS parameters and run GA
max_layers = config.getint('NAS', 'max_layers')
max_iterations = int(config['GA']['max_iterations'])
population_size = int(config['GA']['population_size'])
log_path = str(config['GA']['logs_dir_GA'])
mating_pool_cutoff = float(config['GA']['mating_pool_cutoff'])
mutation_probability = float(config['GA']['mutation_probability'])
############################################
print(f"Max Layers: {max_layers}")
print(f"Max Iterations: {max_iterations}")
print(f"Population Size: {population_size}")
print(f"Log Path: {log_path}")
print(f"Mating Pool Cutoff: {mating_pool_cutoff}")
print(f"Mutation Probability: {mutation_probability}")

Max Layers: 10
Max Iterations: 4
Population Size: 8
Log Path: ./logs/GA_logs
Mating Pool Cutoff: 0.8
Mutation Probability: 0.1


In [4]:
# Define population
from pynattas.builder.population import Population

pop = Population(max_layers=max_layers, n_individuals=population_size)
print(f"Initial Population: {len(pop.population)}")
print(f"Current Generation: {pop.iteration}")

[x.architecture for x in pop.population]

Initial Population: 8
Current Generation: 0


['LOaln1|Pa|Ldo2agn1|PM|LRr3aln1|PM|Ldo3arn1|PM|Leo2k3s1p0aln1|PI|Lne6aln1|PM|LPe4aln1|PI|Ldo3aln1|PI|Ldo2agn1|Pa|Lne6aln1|Pa|HC||',
 'Lbo2k4s2p2aln1|PM|LPe3agn1|PI|Leo4k3s1p1aln1|PI|LPe3aln1|PM|LRr3aln1|Pa|Lne4agn1|PI|LPe3aln1|Pa|Leo4k3s2p0agn1|PI|Lco3k3s1p0agn1|PI|HC||',
 'Ldo4arn1|PM|LRr4agn1|PI|Ldo2agn1|Pa|Leo2k4s2p0aln1|Pa|LPe3aln1|PI|LOagn1|PM|LRr3agn1|Pa|Ldo2arn1|Pa|Leo3k5s2p0agn1|PI|Leo3k3s1p0arn1|PM|HC||',
 'LPe4aln1|PM|Lco3k5s2p1arn1|PI|LRr3agn1|PM|LOaln1|PM|Ldo4arn1|Pa|HC||',
 'Ldo2agn1|Pa|Leo2k5s2p0agn1|PM|Lne6aln1|PM|Lne3arn1|PI|HC||',
 'LPe4aln1|Pa|Lco3k3s2p2agn1|Pa|Ldo2agn1|PI|Lco2k3s1p1aln1|PM|Ldo2arn1|Pa|Ldo3arn1|Pa|Lme6aln1|Pa|HC||',
 'Leo4k5s1p2arn1|Pa|Lco3k4s2p0agn1|PI|Lme3arn1|PM|Lco3k5s1p2arn1|PI|Ldo4agn1|PM|Lme4aln1|PM|Lne5arn1|Pa|Lme5arn1|PM|LRr3arn1|Pa|Ldo2aln1|PM|HC||',
 'LRr3aln1|Pa|LPe3agn1|PI|Lme3aln1|Pa|HC||']

In [9]:
from pynattas.builder.netBuilder import GenericNetwork 
from pynattas.utils import layerCoder

idx = 2
parsed_layers = layerCoder.parse_architecture_code(pop.population[idx].architecture)
parsed_layers

[{'layer_type': 'DenseNetBlock',
  'out_channels_coefficient': '4',
  'activation': 'ReLU',
  'num_blocks': '1'},
 {'layer_type': 'MaxPool'},
 {'layer_type': 'ResNetBlock',
  'reduction_factor': '4',
  'activation': 'GELU',
  'num_blocks': '1'},
 {'layer_type': 'Identity'},
 {'layer_type': 'DenseNetBlock',
  'out_channels_coefficient': '2',
  'activation': 'GELU',
  'num_blocks': '1'},
 {'layer_type': 'AvgPool'},
 {'layer_type': 'ConvSE',
  'out_channels_coefficient': '2',
  'kernel_size': '4',
  'stride': '2',
  'padding': '0',
  'activation': 'LeakyReLU',
  'num_blocks': '1'},
 {'layer_type': 'AvgPool'},
 {'layer_type': 'CSPMBConvBlock',
  'expansion_factor': '3',
  'activation': 'LeakyReLU',
  'num_blocks': '1'},
 {'layer_type': 'Identity'},
 {'layer_type': 'CSPConvBlock', 'activation': 'GELU', 'num_blocks': '1'},
 {'layer_type': 'MaxPool'},
 {'layer_type': 'ResNetBlock',
  'reduction_factor': '3',
  'activation': 'GELU',
  'num_blocks': '1'},
 {'layer_type': 'AvgPool'},
 {'layer_ty

In [10]:
input_channels = 3
input_height = 224
input_width = 224
num_classes = 2

Net = GenericNetwork(
        parsed_layers, 
        input_channels,
        input_height,
        input_width,
        num_classes,
)

AttributeError: 'GenericNetwork' object has no attribute 'outchannels'

In [41]:
import re

segment = 'Lco3k4s2p1agn1'
##########################
params = segment[2:]  # Parameters portion of the segment

# Extract alphanumeric pairs, keeping in mind cases like 'agn1'
param_pairs = re.findall(r'[a-zA-Z]+[0-9]*', params)

segment_info = {}
for pair in param_pairs:
    # Special case for 'agn1': manually split 'a:g' and 'n:1'
    if pair.startswith('a'):
        key_value_pairs = [(pair[0], pair[1]), (pair[2], pair[3])]
    else:
        key_value_pairs = [(pair[0], pair[1:])]

    segment_info.update({key: value for key, value in key_value_pairs})

In [6]:
# from pynattas.optimizers import ga

In [4]:
# nas_result = pnas.optimizers.ga.ga_optimizer(
#     max_layers=max_layers,
#     max_iter=max_iterations,
#     n_individuals=population_size,
#     mating_pool_cutoff=mating_pool_cutoff,
#     mutation_probability=mutation_probability,
#     logs_directory=log_path,
#     task=task,
# )

AttributeError: module 'pynattas.optimizers' has no attribute 'ga'